In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm_notebook as tqdm
import psycopg2  
from peewee import *
from playhouse.postgres_ext import PostgresqlExtDatabase, JSONField, ArrayField
from playhouse.shortcuts import model_to_dict


In [ ]:
with open("data_discr_ganre_arr_word2int","r") as f:
    dataset = json.load(f)
with open("vocab","r") as f:
    vocab = json.load(f)

In [ ]:
dataset[0]

In [ ]:
dataset=dataset[:1000]

In [ ]:
row=[0 for i in range(len(vocab))]

for d in tqdm(dataset):
    r=row.copy()
    for w in d["discr"]:
        r[w]+=1
    d["discr"]=r

In [ ]:
sum(dataset[0]["discr"])

In [ ]:
X=[d["discr"] for d in dataset]
Y=[d["ganre"][0] for d in dataset]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.1, random_state=42)

In [ ]:
from catboost import Pool , CatBoostClassifier
pool=Pool(X_train,y_train)
clf = CatBoostClassifier(iterations=1200, 
                            learning_rate=0.01,
                            depth=5,
#                             class_weights=[1,1.5],
                            random_seed=7)
clf.fit(pool,verbose=False,
                            plot=True)

In [ ]:
from sklearn import metrics
# (clf.predict_proba(X_valid)[:, 1]-y_valid).mean()
pred= clf.predict_proba(X_valid)[:, 1]

In [ ]:
auc = metrics.roc_auc_score(y_valid, pred)

In [ ]:
auc

In [ ]:
clf.predict_proba(X_valid)[:, 1]

In [ ]:
import numpy as np
import scipy.sparse
import pickle
import xgboost as xgb


# specify parameters via map, definition are same as c++ version
param = {'max_depth':6, 'eta':1, 'silent':1, 'objective':'binary:logistic'}

# specify validations set to watch performance
watchlist = [(dtest, 'eval'), (dtrain, 'train')]
num_round = 2
bst = xgb.train(param, dtrain, num_round, watchlist)

In [ ]:
param = {'max_depth': 2, 'eta': 1, 'silent': 1}


In [ ]:
import pickle
pickle.dump(gbm, open("gbm", "wb"))
